# Cleaning Data for Comp 3022


In [36]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [37]:
df = pd.read_csv('MC1/MC1Reports.csv')
df


,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,2020-04-08 17:50:00,10.0,6.0,10.0,3.0,8.0,NaN,1
1,2020-04-09 13:50:00,2.0,10.0,0.0,8.0,4.0,0.0,1
2,2020-04-09 00:20:00,7.0,10.0,10.0,9.0,10.0,0.0,1
3,2020-04-08 17:25:00,1.0,1.0,2.0,10.0,7.0,NaN,1
4,2020-04-08 02:50:00,9.0,7.0,1.0,6.0,9.0,NaN,1
...,...,...,...,...,...,...,...,...
83065,2020-04-10 02:30:00,9.0,10.0,10.0,NaN,7.0,2.0,8
83066,2020-04-10 02:30:00,8.0,10.0,10.0,NaN,7.0,1.0,8
83067,2020-04-09 16:45:00,10.0,9.0,10.0,NaN,8.0,1.0,8
83068,2020-04-09 16:55:00,8.0,8.0,9.0,NaN,7.0,0.0,8


In [38]:
# create a boolean mask for null values
mask = df.isnull().any(axis=1)

# subset the DataFrame using the mask
null_rows = df[mask]
null_rows

,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,2020-04-08 17:50:00,10.0,6.0,10.0,3.0,8.0,NaN,1
3,2020-04-08 17:25:00,1.0,1.0,2.0,10.0,7.0,NaN,1
4,2020-04-08 02:50:00,9.0,7.0,1.0,6.0,9.0,NaN,1
5,2020-04-09 05:30:00,2.0,7.0,3.0,10.0,10.0,NaN,1
6,2020-04-08 04:30:00,2.0,10.0,1.0,2.0,8.0,NaN,1
...,...,...,...,...,...,...,...,...
83065,2020-04-10 02:30:00,9.0,10.0,10.0,NaN,7.0,2.0,8
83066,2020-04-10 02:30:00,8.0,10.0,10.0,NaN,7.0,1.0,8
83067,2020-04-09 16:45:00,10.0,9.0,10.0,NaN,8.0,1.0,8
83068,2020-04-09 16:55:00,8.0,8.0,9.0,NaN,7.0,0.0,8


In [39]:
null_totals = df.isnull().sum(axis=0)
null_totals

time                     0
sewer_and_water        171
power                    0
roads_and_bridges        0
medical              47441
buildings              170
shake_intensity      12144
location                 0
dtype: int64

# Performing median imputation on shake intensity column


In [40]:
si_notnull = df[df['shake_intensity'].notnull()]
si_null = df[df['shake_intensity'].isnull()]

In [41]:
si_notnull['location'].describe()

count    70926.000000
mean         8.812044
std          4.999315
min          1.000000
25%          3.000000
50%          8.000000
75%         13.000000
max         19.000000
Name: location, dtype: float64

In [42]:
medians = si_notnull.groupby('location')['shake_intensity'].median()
df.head()

,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,2020-04-08 17:50:00,10.0,6.0,10.0,3.0,8.0,NaN,1
1,2020-04-09 13:50:00,2.0,10.0,0.0,8.0,4.0,0.0,1
2,2020-04-09 00:20:00,7.0,10.0,10.0,9.0,10.0,0.0,1
3,2020-04-08 17:25:00,1.0,1.0,2.0,10.0,7.0,NaN,1
4,2020-04-08 02:50:00,9.0,7.0,1.0,6.0,9.0,NaN,1


In [43]:
for i, row in df[df['shake_intensity'].isna()].iterrows():
    df.loc[i, 'shake_intensity'] = medians[df.loc[i, 'location']]

In [44]:
null_totals = df.isnull().sum(axis=0)
null_totals

time                     0
sewer_and_water        171
power                    0
roads_and_bridges        0
medical              47441
buildings              170
shake_intensity          0
location                 0
dtype: int64

In [45]:
df.head()

,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,2020-04-08 17:50:00,10.0,6.0,10.0,3.0,8.0,1.0,1
1,2020-04-09 13:50:00,2.0,10.0,0.0,8.0,4.0,0.0,1
2,2020-04-09 00:20:00,7.0,10.0,10.0,9.0,10.0,0.0,1
3,2020-04-08 17:25:00,1.0,1.0,2.0,10.0,7.0,1.0,1
4,2020-04-08 02:50:00,9.0,7.0,1.0,6.0,9.0,1.0,1


# Performing median imputation on Buildings

In [46]:
build_notnull = df[df['buildings'].notnull()]
build_null = df[df['buildings'].isnull()]

build_null


,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
4295,2020-04-08 12:25:00,NaN,1.0,3.0,NaN,NaN,5.0,7
4296,2020-04-10 20:55:00,NaN,10.0,0.0,NaN,NaN,5.0,7
4297,2020-04-06 07:00:00,NaN,4.0,9.0,NaN,NaN,5.0,7
4298,2020-04-07 10:35:00,NaN,0.0,8.0,NaN,NaN,5.0,7
4299,2020-04-06 13:20:00,NaN,4.0,8.0,NaN,NaN,5.0,7
...,...,...,...,...,...,...,...,...
70524,2020-04-09 16:45:00,NaN,7.0,7.0,NaN,NaN,6.0,7
70525,2020-04-09 16:15:00,NaN,7.0,7.0,NaN,NaN,4.0,7
70526,2020-04-09 17:00:00,NaN,8.0,7.0,NaN,NaN,4.0,7
70527,2020-04-09 17:30:00,NaN,7.0,7.0,NaN,NaN,4.0,7


In [47]:
b_medians = build_notnull.groupby('location')['buildings'].median()

for i, row in df[df['buildings'].isna()].iterrows():
    df.loc[i, 'buildings'] = b_medians[df.loc[i, 'location']]

In [48]:
null_totals = df.isnull().sum(axis=0)
null_totals

time                     0
sewer_and_water        171
power                    0
roads_and_bridges        0
medical              47441
buildings                0
shake_intensity          0
location                 0
dtype: int64

# Median imputation for Medical

In [49]:
df
null_totals = df.isnull().sum(axis=0)
null_totals
#Shows all the remaining null values

time                     0
sewer_and_water        171
power                    0
roads_and_bridges        0
medical              47441
buildings                0
shake_intensity          0
location                 0
dtype: int64

In [50]:
#Double checking median values
medical_notnull = df[df['medical'].notnull()]
medical_null = df[df['medical'].isnull()]



#Setting the medians for each location
m_medians = build_notnull.groupby('location')['medical'].median()

medical_null

,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
382,2020-04-10 10:30:00,10.0,1.0,4.0,NaN,5.0,2.0,2
383,2020-04-09 12:10:00,7.0,6.0,6.0,NaN,9.0,2.0,2
384,2020-04-06 08:10:00,10.0,10.0,8.0,NaN,3.0,0.0,2
385,2020-04-08 12:05:00,0.0,5.0,2.0,NaN,4.0,2.0,2
386,2020-04-07 10:25:00,6.0,0.0,1.0,NaN,10.0,2.0,2
...,...,...,...,...,...,...,...,...
83065,2020-04-10 02:30:00,9.0,10.0,10.0,NaN,7.0,2.0,8
83066,2020-04-10 02:30:00,8.0,10.0,10.0,NaN,7.0,1.0,8
83067,2020-04-09 16:45:00,10.0,9.0,10.0,NaN,8.0,1.0,8
83068,2020-04-09 16:55:00,8.0,8.0,9.0,NaN,7.0,0.0,8


In [53]:
m_medians

#Location 7 is null, this indicates there is no medical buildings within this vicinity (Wilson Forrest)

location
1     4.0
2     4.0
3     7.0
4     6.0
5     1.0
6     2.0
7     NaN
8     5.0
9     5.0
10    5.0
11    4.0
12    5.0
13    4.0
14    4.0
15    6.0
16    4.0
17    4.0
18    4.0
19    5.5
Name: medical, dtype: float64

In [62]:
for i, row in df[df['medical'].isna()].iterrows():
    df.loc[i, 'medical'] = m_medians[df.loc[i, 'location']]
    
df = df

In [63]:
null_totals = df.isnull().sum(axis=0)
null_totals

time                   0
sewer_and_water      171
power                  0
roads_and_bridges      0
medical              172
buildings              0
shake_intensity        0
location               0
dtype: int64

# Location 7 is a forrest, there is no hospital nearby


## Going to set null values to zero (As theres no intensity of damage)

In [79]:
med_null = df[df['medical'].isnull()]
for i, row in df[df['medical'].isna()].iterrows():
    df.loc[i, 'medical'] = 0.0
    
null_totals = df.isnull().sum(axis=0)
null_totals

time                 0
sewer_and_water      0
power                0
roads_and_bridges    0
medical              0
buildings            0
shake_intensity      0
location             0
dtype: int64

# Median Impiutaion For Sewer  (171 null)

In [64]:
sew_notnull = df[df['sewer_and_water'].notnull()]
sew_null = df[df['sewer_and_water'].isnull()]

sew_null

sw_medians = build_notnull.groupby('location')['sewer_and_water'].median()
sw_medians

for i, row in df[df['sewer_and_water'].isna()].iterrows():
    df.loc[i, 'sewer_and_water'] = sw_medians[df.loc[i, 'location']]

In [67]:
null_totals = df.isnull().sum(axis=0)
null_totals

#Remains the same as there is no sewage system in forrest

time                   0
sewer_and_water      171
power                  0
roads_and_bridges      0
medical              172
buildings              0
shake_intensity        0
location               0
dtype: int64

## Going to set null values to zero (As theres no intensity of damage)

In [80]:
sew_null = df[df['sewer_and_water'].isnull()]
for i, row in df[df['sewer_and_water'].isna()].iterrows():
    df.loc[i, 'sewer_and_water'] = 0.0
    
null_totals = df.isnull().sum(axis=0)
null_totals

time                 0
sewer_and_water      0
power                0
roads_and_bridges    0
medical              0
buildings            0
shake_intensity      0
location             0
dtype: int64

# Checking Outliers for location 7

In [83]:
location_seven = df[df['location'] == 7.0]

seven_sewer_notnull = location_seven[location_seven['sewer_and_water']!= 0.0]
seven_medical_notnull = location_seven[location_seven['medical']!= 0.0]

seven_sewer_notnull

,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
25845,2020-04-08 09:30:00,10.0,9.0,6.0,0.0,7.0,5.0,7
25898,2020-04-08 09:20:00,7.0,7.0,5.0,0.0,7.0,5.0,7


In [84]:
seven_medical_notnull

,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
57351,2020-04-08 13:15:00,0.0,8.0,7.0,6.0,7.0,-0.0,7


## Deciding to keep outliers but acknowledge

In [86]:
#Remains no more null values
null_totals = df.isnull().sum(axis=0)
null_totals

time                 0
sewer_and_water      0
power                0
roads_and_bridges    0
medical              0
buildings            0
shake_intensity      0
location             0
dtype: int64

# Dataset without null values or duplicates

In [87]:
df.head()

,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,2020-04-08 17:50:00,10.0,6.0,10.0,3.0,8.0,1.0,1
1,2020-04-09 13:50:00,2.0,10.0,0.0,8.0,4.0,0.0,1
2,2020-04-09 00:20:00,7.0,10.0,10.0,9.0,10.0,0.0,1
3,2020-04-08 17:25:00,1.0,1.0,2.0,10.0,7.0,1.0,1
4,2020-04-08 02:50:00,9.0,7.0,1.0,6.0,9.0,1.0,1


In [91]:
df.to_csv('Cleaned_responses_medianImputation.csv', index = False)